## Loan Deliqency

In [1]:
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')
from sklearn.ensemble import RandomForestClassifier

In [2]:
train = pd.read_csv("train.csv")

In [3]:
train.head()

,loan_id,source,financial_institution,interest_rate,unpaid_principal_bal,loan_term,origination_date,first_payment_date,loan_to_value,number_of_borrowers,...,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13
0,268055008619,Z,"Turner, Baldwin and Rhodes",4.250,214000,360,2012-03-01,05/2012,95,1.0,...,0,0,0,1,0,0,0,0,0,1
1,672831657627,Y,"Swanson, Newton and Miller",4.875,144000,360,2012-01-01,03/2012,72,1.0,...,0,0,0,0,0,0,0,1,0,1
2,742515242108,Z,Thornton-Davis,3.250,366000,180,2012-01-01,03/2012,49,1.0,...,0,0,0,0,0,0,0,0,0,1
3,601385667462,X,OTHER,4.750,135000,360,2012-02-01,04/2012,46,2.0,...,0,0,0,0,0,1,1,1,1,1
4,273870029961,X,OTHER,4.750,124000,360,2012-02-01,04/2012,80,1.0,...,3,4,5,6,7,8,9,10,11,1


In [4]:
y = pd.DataFrame(train['m13'])
train.isnull().sum()

loan_id                     0
source                      0
financial_institution       0
interest_rate               0
unpaid_principal_bal        0
loan_term                   0
origination_date            0
first_payment_date          0
loan_to_value               0
number_of_borrowers         0
debt_to_income_ratio        0
borrower_credit_score       0
loan_purpose                0
insurance_percent           0
co-borrower_credit_score    0
insurance_type              0
m1                          0
m2                          0
m3                          0
m4                          0
m5                          0
m6                          0
m7                          0
m8                          0
m9                          0
m10                         0
m11                         0
m12                         0
m13                         0
dtype: int64

In [5]:
print(train.shape)

(116058, 29)


In [6]:
train["m13"].value_counts()

0    115422
1       636
Name: m13, dtype: int64

In [7]:
train.dtypes

loan_id                       int64
source                       object
financial_institution        object
interest_rate               float64
unpaid_principal_bal          int64
loan_term                     int64
origination_date             object
first_payment_date           object
loan_to_value                 int64
number_of_borrowers         float64
debt_to_income_ratio        float64
borrower_credit_score       float64
loan_purpose                 object
insurance_percent           float64
co-borrower_credit_score    float64
insurance_type              float64
m1                            int64
m2                            int64
m3                            int64
m4                            int64
m5                            int64
m6                            int64
m7                            int64
m8                            int64
m9                            int64
m10                           int64
m11                           int64
m12                         

In [8]:
X1 = train.drop(['loan_id', 'first_payment_date','origination_date','m13','insurance_percent','insurance_type'],axis=1)


Here by label encoder I convert object values to numerical values 

In [9]:
from sklearn.preprocessing import LabelEncoder
lm = LabelEncoder()
a  = ['financial_institution']
for i in np.arange(len(a)):
    X1[a[i]] = lm.fit_transform(X1[a[i]])
    


By one hot encoding I convert 'source','loan_purpose' into dummy variables

In [10]:
sourc = pd.get_dummies(X1['source'])
LoanP = pd.get_dummies(X1['loan_purpose'])
X1 = pd.concat([X1,LoanP],axis = 1)
X1 = X1.drop(['source','loan_purpose'],axis=1)


In [11]:
print(X1.shape)


(116058, 24)


In [12]:
X1.head()

,financial_institution,interest_rate,unpaid_principal_bal,loan_term,loan_to_value,number_of_borrowers,debt_to_income_ratio,borrower_credit_score,co-borrower_credit_score,m1,...,m6,m7,m8,m9,m10,m11,m12,A23,B12,C86
0,18,4.250,214000,360,95,1.0,22.0,694.0,0.0,0,...,0,1,0,0,0,0,0,0,0,1
1,15,4.875,144000,360,72,1.0,44.0,697.0,0.0,0,...,0,0,0,0,0,1,0,0,1,0
2,17,3.250,366000,180,49,1.0,33.0,780.0,0.0,0,...,0,0,0,0,0,0,0,0,1,0
3,8,4.750,135000,360,46,2.0,44.0,633.0,638.0,0,...,0,0,0,1,1,1,1,0,1,0
4,8,4.750,124000,360,80,1.0,43.0,681.0,0.0,0,...,5,6,7,8,9,10,11,0,0,1


In [13]:
X1['credit_score'] = X1['borrower_credit_score'] + X1['co-borrower_credit_score']

In [14]:
X1['mean']=X1[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7','m8', 'm9', 'm10', 'm11', 'm12']].mean(axis=1)

In [15]:
X1['sum']=X1[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7','m8', 'm9', 'm10', 'm11', 'm12']].sum(axis=1)

In [16]:
X1['skew']=X1[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7','m8', 'm9', 'm10', 'm11', 'm12']].skew(axis=1)

In [17]:
X1['kurt']=X1[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7','m8', 'm9', 'm10', 'm11', 'm12']].kurt(axis=1)

In [18]:
X1.columns

Index(['financial_institution', 'interest_rate', 'unpaid_principal_bal',
       'loan_term', 'loan_to_value', 'number_of_borrowers',
       'debt_to_income_ratio', 'borrower_credit_score',
       'co-borrower_credit_score', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7',
       'm8', 'm9', 'm10', 'm11', 'm12', 'A23', 'B12', 'C86', 'credit_score',
       'mean', 'sum', 'skew', 'kurt'],
      dtype='object')

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.25, random_state=2)



rft=RandomForestClassifier(random_state=42,max_depth= 10 ,min_samples_split= 5 ,n_estimators= 500,n_jobs=-1, min_samples_leaf=2,  criterion='entropy',class_weight= {0: 0.8, 1:1})
rft.fit(X_train,y_train) 

from sklearn.metrics import f1_score,confusion_matrix

y_pred=rft.predict(X_test)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.47154471544715443


array([[28827,    19],
       [  111,    58]])

In [21]:
threshold = 0.23
y_pred_prob = rft.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.5321100917431193


array([[28775,    71],
       [   82,    87]])

In [22]:
pd.Series(rft.feature_importances_,index=X1.columns).sort_values(ascending=False)

m12                         0.166406
mean                        0.127428
sum                         0.118794
skew                        0.086807
m11                         0.059208
borrower_credit_score       0.052200
credit_score                0.047105
kurt                        0.039991
m10                         0.037493
unpaid_principal_bal        0.032268
interest_rate               0.031493
debt_to_income_ratio        0.031336
m9                          0.027460
loan_to_value               0.025861
co-borrower_credit_score    0.024379
m8                          0.017840
financial_institution       0.015623
m7                          0.011928
loan_term                   0.008845
m5                          0.006658
m6                          0.005672
B12                         0.004467
A23                         0.004396
C86                         0.003695
m4                          0.003525
number_of_borrowers         0.003421
m3                          0.002481
m

In [23]:
X1.columns

Index(['financial_institution', 'interest_rate', 'unpaid_principal_bal',
       'loan_term', 'loan_to_value', 'number_of_borrowers',
       'debt_to_income_ratio', 'borrower_credit_score',
       'co-borrower_credit_score', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7',
       'm8', 'm9', 'm10', 'm11', 'm12', 'A23', 'B12', 'C86', 'credit_score',
       'mean', 'sum', 'skew', 'kurt'],
      dtype='object')

In [24]:
X1=X1.drop(['m1', 'm2', 'm3','A23','B12','C86','number_of_borrowers'],axis=1)

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=2)



rft=RandomForestClassifier(random_state=42,max_depth= 10 ,min_samples_split= 5 ,n_estimators= 500,n_jobs=-1, min_samples_leaf=2,  criterion='entropy',class_weight= {0: 0.8, 1:1})
rft.fit(X_train,y_train) 

from sklearn.metrics import f1_score,confusion_matrix

y_pred=rft.predict(X_test)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.4863013698630136


array([[34597,    22],
       [  128,    71]])

In [45]:
threshold = 0.27
y_pred_prob = rft.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.5469613259668508


array([[34555,    64],
       [  100,    99]])

In [46]:
X1=X1.drop(['m4','m5','m6','m7','loan_term'],axis=1)

In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=2)



rft=RandomForestClassifier(random_state=42,max_depth= 6 ,min_samples_split= 10 ,n_estimators= 500,n_jobs=-1, min_samples_leaf=2,  criterion='entropy',class_weight= {0: 0.8, 1:1})
rft.fit(X_train,y_train) 

from sklearn.metrics import f1_score,confusion_matrix

y_pred=rft.predict(X_test)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.48648648648648646


array([[34594,    25],
       [  127,    72]])

In [67]:
threshold = 0.258
y_pred_prob = rft.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.5459317585301837


array([[34541,    78],
       [   95,   104]])

I hyper tuned my random forest classifier by varying depth between [6,15] and min sample split between[2,5]. I used criterion as "Entropy" because entropy has an edge in some data cases involving a high imbalance. 
I have given class weight { 0:0.8, 1:1 } to minimize 'class1' error which means "It is ok that If I don't give loan to 2-3 good guys but I don't want to give loan to those people who can be loan defaulters".

In [47]:
pd.Series(rft.feature_importances_,index=X1.columns).sort_values(ascending=False)

sum                         0.155960
m12                         0.141925
mean                        0.139978
skew                        0.105815
m11                         0.067038
kurt                        0.056121
m10                         0.055876
borrower_credit_score       0.053828
credit_score                0.044362
m9                          0.037965
debt_to_income_ratio        0.025590
interest_rate               0.024597
unpaid_principal_bal        0.023102
co-borrower_credit_score    0.021036
loan_to_value               0.019750
financial_institution       0.011455
m8                          0.010026
loan_term                   0.005577
dtype: float64

In [68]:
# XGB Classifier
from xgboost import XGBClassifier

classifier = XGBClassifier( learning_rate =0.1,
 n_estimators=91,
 max_depth=32,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=13,
 reg_lambda=5,
# max_delta_step=1,
 alpha=0,
 base_score=0.5,
 seed=1029)

classifier.fit(X_train, y_train)

XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=32,
              min_child_weight=5, missing=None, n_estimators=91, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=5, scale_pos_weight=13, seed=1029,
              silent=None, subsample=0.8, verbosity=1)

In [83]:
threshold = 0.4
y_pred_prob = classifier.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.5295629820051414


array([[34532,    87],
       [   96,   103]])

In [84]:
# XGB Classifier
from xgboost import XGBClassifier

classifier = XGBClassifier( learning_rate =0.1,
 n_estimators=91,
 max_depth=16,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=13,
 reg_lambda=5,
# max_delta_step=1,
 alpha=0,
 base_score=0.5,
 seed=1029)

classifier.fit(X_train, y_train)

XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=16,
              min_child_weight=5, missing=None, n_estimators=91, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=5, scale_pos_weight=13, seed=1029,
              silent=None, subsample=0.8, verbosity=1)

In [98]:
threshold = 0.47
y_pred_prob = classifier.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.544


array([[34545,    74],
       [   97,   102]])

In [214]:
# XGB Classifier
from xgboost import XGBClassifier

classifier = XGBClassifier( learning_rate =0.1,
 n_estimators=100,
 max_depth=20,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.6,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=13,
 reg_lambda=5,
# max_delta_step=1,
 alpha=0,
 base_score=0.5,
 seed=1029)

classifier.fit(X_train, y_train)

XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=20,
              min_child_weight=5, missing=None, n_estimators=100, n_jobs=1,
              nthread=4, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=5, scale_pos_weight=13, seed=1029,
              silent=None, subsample=0.8, verbosity=1)

In [219]:
threshold = 0.41
y_pred_prob = classifier.predict_proba(X_test)[:,1]
y_pred = (y_pred_prob > threshold).astype(int)

print(f1_score(y_test,y_pred))

confusion_matrix(y_test,y_pred)

0.531578947368421


array([[34539,    80],
       [   98,   101]])